In [1]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from dotenv import load_dotenv

load_dotenv()

llm = ChatNVIDIA(model="ai-mixtral-8x7b-instruct", max_tokens=1000) 

/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_statics.py:319: UserWarning: Model ai-mixtral-8x7b-instruct is deprecated. Using mistralai/mixtral-8x7b-instruct-v0.1 instead.
  warnings.warn(


In [4]:


def index_docs(url, splitter, documents, dest_embed_dir) -> None:
    """
    Split the document into chunks and create embeddings for the document

    Args:
        url: Source url for the document.
        splitter: Splitter used to split the document
        documents: list of documents whose embeddings needs to be created
        dest_embed_dir: destination directory for embeddings

    Returns:
        None
    """
    


In [25]:
from os import listdir
from os.path import isfile, join
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import re

# Input for dir_path is './<dir name>'
def batch_document_reader(dir_path):

    # Getting filenames in the directory
    filenames = [f for f in listdir(dir_path) if isfile(join(dir_path, f))]

    documents = []
    for pdf in filenames:
        pdf_reader = PdfReader(dir_path + '/' + pdf)
        
        print(f"Reading document: {pdf}")
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        
        # Remove excess whitespace and newlines
        text = re.sub("\s+", " ", text).strip()

        documents.append(text)
        
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0,
        length_function=len,
    )
    
    texts = text_splitter.create_documents(documents)
    print(len(texts))
        
    return texts, text_splitter

In [26]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
def create_embeddings(documents, splitter, embedding_path):

    # embedding_path = "./embed"
    print(f"Storing embeddings to filepath {embedding_path}")

    embeddings = NVIDIAEmbeddings(model="nvolveqa_40k")

    for document in documents:
        
        texts = splitter.split_text(document.page_content)

        # metadata to attach to document
        metadatas = [document.metadata]

        # create embeddings and add to vector store
        if os.path.exists(embedding_path):
            update = FAISS.load_local(folder_path=embedding_path, embeddings=embeddings, allow_dangerous_deserialization=True)
            update.add_texts(texts, metadatas=metadatas)
            update.save_local(folder_path=embedding_path)
        else:
            docsearch = FAISS.from_texts(texts, embedding=embeddings, metadatas=metadatas)
            docsearch.save_local(folder_path=embedding_path)

    print("Generated embedding successfully")
    return

In [27]:
texts, text_splitter = batch_document_reader("./pdf_samples")
create_embeddings(texts, text_splitter, "./embed")

Reading document: Models.pdf
Reading document: Transport.pdf
Reading document: textbook.pdf
537
Storing embeddings to filepath ./embed
Generated embedding successfully


RUN FROM HERE

In [1]:
# loading the 
from langchain.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
embedding_model = NVIDIAEmbeddings(model="nvolveqa_40k")
docsearch = FAISS.load_local(folder_path="./embed", embeddings=embedding_model, allow_dangerous_deserialization=True)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_statics.py:319: UserWarning: Model nvolveqa_40k is deprecated. Using NV-Embed-QA instead.
  warnings.warn(
/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/embeddings.py:87: UserWarning: Setting truncate="END" for nvolveqa_40k backward compatibility
  warnings.warn(


In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT, QA_PROMPT
from langchain.chains.question_answering import load_qa_chain

from dotenv import load_dotenv

load_dotenv()

llm = ChatNVIDIA(model="ai-mixtral-8x7b-instruct", temperature=0.1, max_tokens=1000, top_p=1.0)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

qa_prompt=QA_PROMPT

doc_chain = load_qa_chain(llm, chain_type="stuff", prompt=QA_PROMPT)

qa = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=docsearch.as_retriever(),
    chain_type="stuff",
    memory=memory,
    combine_docs_chain_kwargs={'prompt': qa_prompt},
)

/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_statics.py:319: UserWarning: Model ai-mixtral-8x7b-instruct is deprecated. Using mistralai/mixtral-8x7b-instruct-v0.1 instead.
  warnings.warn(


In [3]:
query = "What is the Aloha Network?"
result = qa({"question": query})
print(result.get("answer"))

/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


END

In [ ]:
query = "Explain it to a 5 year old."
result = qa({"question": query})
print(result.get("answer"))

In [1]:
from langchain.vectorstores import FAISS
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

embedding_model = NVIDIAEmbeddings(model="nvolveqa_40k")

store = FAISS.load_local("./embed", embedding_model)

retriever = store.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("What is the Aloha Network?")


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/_statics.py:319: UserWarning: Model nvolveqa_40k is deprecated. Using NV-Embed-QA instead.
  warnings.warn(
/Users/jewel/opt/anaconda3/envs/nvidiatest/lib/python3.10/site-packages/langchain_nvidia_ai_endpoints/embeddings.py:87: UserWarning: Setting truncate="END" for nvolveqa_40k backward compatibility
  warnings.warn(


ValueError: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and know that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).